# 0. Initialize

In [0]:
# 作業ディレクトリの指定 任意のディレクトリを指定して下さい
workdir = '/content/drive/My Drive/test_eclaire'

In [0]:
# 使用するFITSファイル等を保存するためにGoogle Driveをマウントする
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import sys
import os
import shutil
import requests
import functools
from glob import glob

from astropy.io import fits
from astropy.wcs import WCS
import cupy  as cp
import numpy as np

import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval
from tqdm.notebook import tqdm

In [ ]:
# Ecliareのインストール
!pip install git+https://github.com/MNiwano/Eclaire

In [0]:
from eclaire import FitsContainer, reduction, fixpix, imalign, imcombine

In [0]:
fitslist = ['raw{:02d}.fits'.format(i) for i in range(10)]

dark = 'dark.fits'
flat = 'flat.fits'
bpmask = 'bpmask.fits'

output = 'combine.fits'

In [0]:
# Google Drive上に作業ディレクトリを作成し、そこへテストデータをダウンロードする
if not os.path.exists(workdir):
  os.mkdir(workdir)

os.chdir(workdir)

url = 'http://sncwall.hp.phys.titech.ac.jp:2388/samplefits/'
for f in tqdm(fitslist+[dark,flat,bpmask]):
    if not os.path.exists(f):
        res = requests.get(url+f, stream=True)
        with open(f, 'wb') as fp:
            shutil.copyfileobj(res.raw, fp)

  # 1. Data Loading
  <b>eclair.FitsContainer</b>クラスを利用して一次処理を行う天体画像を読み込みます。ファイル名のリストを与えると、与えられたリスト内のFITSをカレントディレクトリから探して読み込みます。
  読み込んだ情報のうち、ヘッダは<b>FitsContainer.header</b>に<b>list</b>として、画像データは<b>FitsContainer.data</b>に3次元<b>cupy.ndarray</b>として格納されます。<b>data</b>は第2軸がY軸、第3軸がX軸に対応し、各画像が第1軸方向にスタックされています。

In [0]:
fc = FitsContainer(
    fitslist,
    #wrapper=functools.partial(tqdm,total=len(fitslist)), # プログレスバーの表示
)

#使用する画像に含まれるオーバースキャン領域の除去
fc.data = fc.data[:,2:1022,52:1072]

In [0]:
npdark = fits.getdata(dark)
npflat = fits.getdata(flat)
npbpm = fits.getdata(bpmask)

cpdark = cp.asarray(npdark,dtype='float32')
cpflat = cp.asarray(npflat+(npflat==0.0),dtype='float32')
cpbpm = cp.asarray(npbpm,dtype='float32')

# 2. Calculate shift amount
用意したFITSにはWCSが貼り付けてあるので、これを利用して画像同士の相対位置を求めます。ここではEclaireは特に使用しません。

※ <b>RADECSYS = 'FK5'</b> のため<b>FITSFixedWarning</b>が発生しますが、無視して下さい。

In [0]:
wcs = [WCS(header) for header in fc.header]

xy0 = np.array([[450,450],[450,650],[650,450],[650,650]])
ad0 = wcs[0].wcs_pix2world(xy0,1)
shift = np.empty([len(wcs),2],dtype='f4')
for i, w in enumerate(wcs):
    xy = w.wcs_world2pix(ad0,1)
    shift[i] = (xy0-xy).mean(axis=0)

# 3. Reduction
まず、<b>eclaire.reduction</b>でバイアス、ダーク引き、フラット補正を一度に行います。この関数は、この例で言えば
```
fc.data = (fc.data - cpbias - cpdark) / cpflat
```
という式と等価ですが、関数呼び出しのオーバーヘッドとメモリ使用量がより少ないです。各入力値はbroadcastableである必要があります。(<b>reshape</b>で<b>cpbias</b>の次元を増しているのはこのためです。)
次に<b>eclaire.fixpix</b>でバッドピクセルの補正を行います。この関数はバッドピクセルを周囲のピクセルカウントの平均で上書きします。与えるバッドピクセルマスクは、バッドピクセルの位置の値が非0、それ以外が0である必要があります。

※ <b>%time</b>は実行時間計測のマジックコマンドで、コードの実行には関係ありません。

In [0]:
# ここではヘッダに書かれたバイアスカウントを使用するが、
# マスターバイアスフレームを使用する場合はダークと同じ要領でcupy.ndarrayとして読み込んだものを使用すれば良い。
bias = [f['PEDLEVEL'] for f in fc.header]
cpbias = cp.array(bias,dtype='f4').reshape(-1,1,1)

In [0]:
fc.data = reduction(fc.data, cpbias, cpdark, cpflat)

In [0]:
fc.data = fixpix(fc.data, cpbpm)

# 4. Align
先程求めた相対位置をもとに、画像同士の位置合わせを行います。
<b>shift</b>は2次元のarray-likeでなければならず、第2軸方向の1つ目の値がX座標、2つ目の値がY座標と解釈されます。
第1軸方向の並び順は、第1引数で与えるarrayに於ける画像の並び順と同じである必要があります。

サブピクセルシフト時の補間アルゴリズムのデフォルトは3次スプライン補間です。

In [0]:
fc.data = imalign(
    fc.data, shift,
    #interp='spline3', # 補間アルゴリズムの指定方法 他にneighbor, linear, poly3が選択可能
)

# 5. Combine
画像の重ね合わせを行います。デフォルトのアルゴリズムはsigma-clipped-meanです。

In [0]:
combined = imcombine(
    fc.data, name=output, list=fitslist, overwrite=True,
    #combine='mean' # 重ね合わせ方法の指定 medianが選択可能
    #width=3.0 # クリッピング幅の指定方法
    #iters=5 # イテレーション回数の指定方法 Noneだと収束するまで
)

# 6. Show Result

In [0]:
data = fits.getdata(output)
vrange = ZScaleInterval().get_limits(data)
data.clip(*vrange,out=data)

plt.figure()
plt.imshow(data,origin='lower')
plt.colorbar()
plt.show()